In [1]:
'''测试kitti变换矩阵'''
import numpy as np
from PIL import Image
from PIL import ImageDraw

data_folder = '/media/shitong/passport/SemanticKITTI/sequences/'
dataset = '00/'
calib_path = data_folder + dataset + 'calib.txt'
data = 0
data = str(data).zfill(6)
img_data = calib_path[:-9] + 'image_2/' + data + '.png'
velodyne_data = img_data.replace('image_2', 'velodyne')[:-3] + 'bin'

scan = np.fromfile(velodyne_data, dtype=np.float32).reshape((-1, 4))
xyz = scan[:, 0:3]
distance = np.linalg.norm(xyz, axis=1)

with open(calib_path) as calib_file:
    lines = calib_file.readlines()
    P2 = lines[2].split(':')[1].split(' ')[1:]
    P2 = np.array([float(p) for p in P2]).reshape((3, 4))
    Tr = lines[4].split(':')[1].split(' ')[1:]
    Tr = np.array([float(p) for p in Tr]).reshape((3, 4))
    identity = np.identity(4)
    identity[:3, :4] = Tr
    Tr = identity

mask = xyz[:, 0] > 0
coordinate = xyz[mask]
distance = distance[mask]
cat_array = np.ones((coordinate.shape[0], 1))
coordinate = np.concatenate([coordinate, cat_array], axis=1)
coordinate = (P2 @ Tr @ coordinate.T).T[:, 0:3]
coordinate = (coordinate / coordinate[:, 2:3])[:, 0:2]

H = 376
W = 1241
mask = (coordinate[:, 0] >= 0) * (coordinate[:, 0] < W) * (coordinate[:, 1] > 0) * (coordinate[:, 1] < H)
coordinate = np.fliplr(coordinate[mask].astype(np.int32))
distance = distance[mask]
distance = (distance - distance.min()) / (distance.max() - distance.min()) * 255
proj_img = np.zeros((H, W))
proj_img[coordinate[:, 0], coordinate[:, 1]] = distance.astype(np.uint8)
coordinate = np.fliplr(coordinate).tolist()

rgb_img = Image.open(img_data)
image_draw = ImageDraw.Draw(rgb_img)
for i in range(len(coordinate)):
    image_draw.point(coordinate[i], (distance[i],0,0))
rgb_img.show()

show_img = Image.fromarray(proj_img)
show_img.show()

In [4]:
'''测试kitti变换矩阵'''
import numpy as np
from PIL import Image
from PIL import ImageDraw

data_folder = '/media/shitong/passport/SemanticKITTI/sequences/'
dataset = '00/'
data = 0
data = str(data).zfill(6)
img_data = data_folder + dataset + 'image_2/' + data + '.png'
velodyne_data = img_data.replace('image_2', 'velodyne')[:-3] + 'bin'
calib_path = './calib.txt'

scan = np.fromfile(velodyne_data, dtype=np.float32).reshape((-1, 4))

xyz = scan[:, 0:3]
cat_dim = np.ones((xyz.shape[0], 1))
xyz = np.concatenate([xyz, cat_dim], axis=1)
Tr = np.identity(4)
Tr[2, 3] = -0.205
xyz = (Tr @ xyz.T).T[:, :3]
distance = np.linalg.norm(xyz, axis=1)
pitch = np.arcsin(xyz[:, 2] / distance) / np.pi * 180
yaw = -np.arctan2(xyz[:, 1], -xyz[:, 0]) / np.pi
proj_x = 0.5 * (yaw + 1)
jump_index = np.nonzero((yaw[1:] < 0.2) * (yaw[:-1] > 0.8))[0] + 1
jump = np.zeros_like(proj_x).astype(np.uint32)
jump[jump_index] = 1
jump = np.cumsum(jump, axis=0)
np.savetxt('./pitch.txt', pitch)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
with open(calib_path) as calib_file:
    lines = calib_file.readlines()
    K = lines[0].split(':')[1].split(' ')[1:]
    K = np.array([float(p) for p in K]).reshape((3, 4))
    Tr = np.identity(4)
    read_tr = lines[1].split(':')[1].split(' ')[1:]
    Tr[:3, :4] = np.array([float(p) for p in read_tr]).reshape((3, 4))

Tt = np.array([[0, 0, -1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])
mask = xyz[:, 0] > 0
coordinate = xyz[mask]
distance0 = distance[mask]
cat_array = np.ones((coordinate.shape[0], 1))
coordinate = np.concatenate([coordinate, cat_array], axis=1)
coordinate0 = (K @ Tr @ coordinate.T).T[:, 0:3]
coordinate0 = (coordinate0 / coordinate0[:, 2:3])[:, 0:2]

mask = xyz[:, 1] < 0
coordinate = xyz[mask]
distance1 = distance[mask]
cat_array = np.ones((coordinate.shape[0], 1))
coordinate = np.concatenate([coordinate, cat_array], axis=1)
coordinate1 = (K @ Tt @ Tr @ coordinate.T).T[:, 0:3]
coordinate1 = (coordinate1 / coordinate1[:, 2:3])[:, 0:2]

mask = xyz[:, 0] < 0
coordinate = xyz[mask]
distance2 = distance[mask]
cat_array = np.ones((coordinate.shape[0], 1))
coordinate = np.concatenate([coordinate, cat_array], axis=1)
coordinate2 = (K @ Tt @ Tt @ Tr @ coordinate.T).T[:, 0:3]
coordinate2 = (coordinate2 / coordinate2[:, 2:3])[:, 0:2]

mask = xyz[:, 1] > 0
coordinate = xyz[mask]
distance3 = distance[mask]
cat_array = np.ones((coordinate.shape[0], 1))
coordinate = np.concatenate([coordinate, cat_array], axis=1)
coordinate3 = (K @ Tt @ Tt @ Tt @ Tr @ coordinate.T).T[:, 0:3]
coordinate3 = (coordinate3 / coordinate3[:, 2:3])[:, 0:2]

H = 376
W = 1241
mask = (coordinate0[:, 0] >= 0) * (coordinate0[:, 0] < W) * (coordinate0[:, 1] > 0) * (coordinate0[:, 1] < H)
coordinate0 = np.fliplr(coordinate0[mask].astype(np.uint32))
distance0 = distance0[mask]
distance0 = (distance0 - distance0.min()) / (distance0.max() - distance0.min()) * 255

mask = (coordinate1[:, 0] >= 0) * (coordinate1[:, 0] < W) * (coordinate1[:, 1] > 0) * (coordinate1[:, 1] < H)
coordinate1 = np.fliplr(coordinate1[mask].astype(np.uint32))
distance1 = distance1[mask]
distance1 = (distance1 - distance1.min()) / (distance1.max() - distance1.min()) * 255

mask = (coordinate2[:, 0] >= 0) * (coordinate2[:, 0] < W) * (coordinate2[:, 1] > 0) * (coordinate2[:, 1] < H)
coordinate2 = np.fliplr(coordinate2[mask].astype(np.uint32))
distance2 = distance2[mask]
distance2 = (distance2 - distance2.min()) / (distance2.max() - distance2.min()) * 255

mask = (coordinate3[:, 0] >= 0) * (coordinate3[:, 0] < W) * (coordinate3[:, 1] > 0) * (coordinate3[:, 1] < H)
coordinate3 = np.fliplr(coordinate3[mask].astype(np.uint32))
distance3 = distance3[mask]
distance3 = (distance3 - distance3.min()) / (distance3.max() - distance3.min()) * 255

proj_img = np.zeros((H, 4 * W))
proj_img[coordinate0[:, 0], coordinate0[:, 1]] = distance0
proj_img[coordinate1[:, 0], coordinate1[:, 1] + W] = distance1
proj_img[coordinate2[:, 0], coordinate2[:, 1] + 2 * W] = distance2
proj_img[coordinate3[:, 0], coordinate3[:, 1] + 3 * W] = distance3

# coordinate0 = np.fliplr(coordinate0).tolist()

# show_img = Image.fromarray(proj_img)
# show_img.show()

In [ ]:
insertlines = 64
fov_up = 2.25
fov_down = 2.25 - 0.5 * insertlines
points_num = 1024 // 4

fai_range = []
for i in range(jump.max()):
    fai_range.append((np.mean(pitch[jump == i]) + 0.2) / 180 * np.pi)
denser_mask = [[np.cos(fai) * np.sin(theta), -np.sin(fai), np.cos(fai) * np.cos(theta)] for theta in np.linspace(-np.pi / 4, np.pi / 4, points_num)
               for fai in fai_range]
denser_mask = np.array(denser_mask)
denser_mask = (denser_mask / denser_mask[:, 2:3])
cat_ones = np.ones((denser_mask.shape[0], 1))
denser_mask = np.concatenate([denser_mask, cat_ones], axis=1)

denser_coordinate = ((K @ denser_mask.T).T)[:, :2].astype(np.uint32)
mask = (denser_coordinate[:, 0] >= 0) * (denser_coordinate[:, 0] < W) * (denser_coordinate[:, 1] > 0) * (denser_coordinate[:, 1] < H)
denser_coordinate = denser_coordinate[mask]

proj_img[denser_coordinate[:, 1], denser_coordinate[:, 0]] = 500.
proj_img[denser_coordinate[:, 1], denser_coordinate[:, 0] + W] = 500.
proj_img[denser_coordinate[:, 1], denser_coordinate[:, 0] + 2 * W] = 500.
proj_img[denser_coordinate[:, 1], denser_coordinate[:, 0] + 3 * W] = 500.

show_img = Image.fromarray(proj_img)
show_img.show()
